출처: https://blog.breezymind.com/2018/03/02/sklearn-feature_extraction-text-2/

In [4]:

import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
np.random.seed(0)

from konlpy.tag import Okt
twitter = Okt()

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# tokenizer : 문장에서 색인어 추출을 위해 명사,동사,알파벳,숫자 정도의 단어만 뽑아서 normalization, stemming 처리하도록 함
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number"], stopword=['수','퀄리티','도시','분','전문','스타','년','원']):
    return [
        word for word, tag in twitter.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

# 테스트 문장
df = pd.read_csv("word2vec_wrangling.csv")
df.shape

(61, 2)

In [7]:
df.head()

,exercise_name,Content_txt
0,PT,"💯 What I try to educate my clients around, doe..."
1,검도,#20200115\n저녁 초대!\n와인잔 속에 비치는\n모든 것들이 화려한\n도심속...
2,기구필라테스,#오늘의동작\n캐딜락 동작의 완성 '행잉'\n⠀\n중력을 이용해 척추를 늘려주고\n...
3,다빈치바디보드,#mbn생생정보마당 \n#고투\n#고투GX\n#다빈치바디보드\n#생방송 #GOTOL...
4,드럼스틱,#드럼스틱 #고무팁 #테크라스틱 #전자드럼용스틱\n\n1. 전자드럼타격시 덜 시끄럽...


In [15]:
import re

def preprocessing(text):
    # 개행문자 제거
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [16]:
%time rawdata = df['Content_txt'].apply(preprocessing)

CPU times: user 1.05 s, sys: 7.14 ms, total: 1.05 s
Wall time: 1.07 s


In [17]:
vectorize = CountVectorizer(
    tokenizer=tokenizer, 
    min_df=10    # 예제로 보기 좋게 1번 정도만 노출되는 단어들은 무시하기로 했다
                # min_df = 0.01 : 문서의 1% 미만으로 나타나는 단어 무시
                # min_df = 10 : 문서에 10개 미만으로 나타나는 단어 무시
                # max_df = 0.80 : 문서의 80% 이상에 나타나는 단어 무시
                # max_df = 10 : 10개 이상의 문서에 나타나는 단어 무시
)
 
# 문장에서 노출되는 feature(특징이 될만한 단어) 수를 합한 Document Term Matrix(이하 DTM) 을 리턴한다
X = vectorize.fit_transform(rawdata)
 
print(
    'fit_transform, (sentence {}, feature {})'.format(X.shape[0], X.shape[1])
)
# fit_transform, (sentence 5, feature 7)
 
print(type(X))
# <class 'scipy.sparse.csr.csr_matrix'>
 
print(X.toarray())

# [[0, 1, 2, 0, 0, 0, 1],
# [0, 1, 1, 0, 0, 0, 2],
# [1, 0, 0, 2, 1, 1, 0],
# [1, 0, 0, 1, 0, 0, 0],
# [0, 0, 0, 3, 1, 1, 0]]
 
# 문장에서 뽑아낸 feature 들의 배열
features = vectorize.get_feature_names()

fit_transform, (sentence 61, feature 7708)
<class 'scipy.sparse.csr.csr_matrix'>
[[ 11  61  35 ...   2   0   0]
 [  0   0   0 ...   0   2   0]
 [  0   0   1 ...  19   0   0]
 ...
 [  1   1   1 ...   0   0   0]
 [  0   0   2 ...   9   0   0]
 [  0   0   0 ...   1 195   0]]


In [18]:
# 검색 문장에서 feature를 뽑아냄
# 즉 검색은 문장 형태로 입력하지만, search keyword는 Okt Tokenizer을 거쳐서 추출되는 것임
srch=[t for t in tokenizer('척추에 좋은 운동') if t in features]
print(srch)
# ['1987', '대통령']
 
# dtm 에서 검색하고자 하는 feature만 뽑아낸다.
srch_dtm = np.asarray(X.toarray())[:, [
    # vectorize.vocabulary_.get 는 특정 feature 가 dtm 에서 가지고 있는 index값을 리턴한다
    vectorize.vocabulary_.get(i) for i in srch
]]

['척추', '운동']
